In [1]:
import os
import sys
import csv
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
import math
import copy
import random

sys.path.append('./backbones/asrf')
from libs.optimizer import get_optimizer
from libs.loss_fn import ActionSegmentationLoss, BoundaryRegressionLoss
from libs.class_weight import get_class_weight, get_pos_weight
from libs.dataset import ActionSegmentationDataset, collate_fn
from libs.transformer import TempDownSamp, ToTensor
from libs.helper import train, validate, evaluate
from libs.checkpoint import resume, save_checkpoint

from src.utils import eval_txts, load_meta
from src.predict import predict_backbone
import configs.mgru_config as cfg
from src.mgru import mGRU

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic=True

In [3]:
device = 'cuda'

In [4]:
dataset = 'breakfast'     # choose from gtea, 50salads, breakfast
split = 2            # gtea : 1~4, 50salads : 1~5, breakfast : 1~4
model_name = 'mgru'  # always "mgru" in this notebook

In [5]:
actions_dict, \
num_actions, \
gt_path, \
features_path, \
vid_list_file, \
vid_list_file_tst, \
sample_rate,\
model_dir,\
result_dir, \
record_dir = load_meta(cfg.dataset_root, cfg.model_root, cfg.result_root, cfg.record_root, dataset, split, model_name)

Created :./model/mgru/breakfast/split_2
Created :./result/mgru/breakfast/split_2
Created :./record/mgru/breakfast


In [6]:
train_data = ActionSegmentationDataset(
        dataset,
        transform=Compose([ToTensor(), TempDownSamp(sample_rate)]),
        mode="trainval" if not cfg.param_search else "training",
        split=split,
        dataset_dir=cfg.dataset_root,
        csv_dir=cfg.csv_dir,
    )
train_loader = DataLoader(
        train_data,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=4,
        drop_last=True if cfg.batch_size > 1 else False,
        collate_fn=collate_fn,
    )

In [7]:
model = mGRU(num_layers=cfg.gru_layers, 
             feat_dim=cfg.gru_hidden_dim, 
             inp_dim=cfg.in_channel, 
             out_dim=num_actions)
model.to(device)

mGRU(
  (in_linear): Linear(in_features=2048, out_features=512, bias=True)
  (gru): GRU(512, 512, num_layers=3, batch_first=True, bidirectional=True)
  (out_linear): Linear(in_features=1024, out_features=48, bias=True)
)

In [8]:
optimizer = get_optimizer(
        'Adam',
        model,
        cfg.learning_rate,
        momentum=cfg.momentum,
        dampening=cfg.dampening,
        weight_decay=cfg.weight_decay,
        nesterov=cfg.nesterov,
)

Adam will be used as an optimizer.


In [9]:
def train(train_loader, model, optimizer, epoch, device):
    model.train()
    normal_ce = nn.CrossEntropyLoss()
    total_loss = 0.0
    for idx, sample in enumerate(train_loader):
        x = sample['feature']
        t = sample['label']
        
        x, t = x.to(device), t.to(device)
        
        B, L, D = x.shape
        
        pred = model(x)
        
        loss = 0.0
        loss += normal_ce(pred[0].permute(1, 0), t[0])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss / len(train_loader)
        
    return total_loss.item()

In [10]:
for epoch in range(cfg.max_epoch):
    # training
    train_loss = train(
        train_loader,
        model,
        optimizer,
        epoch,
        device,
    )
    torch.save(model.state_dict(), os.path.join(model_dir, "epoch-"+str(epoch+1)+".model"))
    print("epoch: {}\tlr: {:.4f}\ttrain loss: {:.4f}".format(epoch+1, optimizer.param_groups[0]["lr"], train_loss))


epoch: 1	lr: 0.0005	train loss: 2.6336
epoch: 2	lr: 0.0005	train loss: 1.7763
epoch: 3	lr: 0.0005	train loss: 1.3988
epoch: 4	lr: 0.0005	train loss: 1.2004
epoch: 5	lr: 0.0005	train loss: 0.9988
epoch: 6	lr: 0.0005	train loss: 0.8794
epoch: 7	lr: 0.0005	train loss: 0.7830
epoch: 8	lr: 0.0005	train loss: 0.7037
epoch: 9	lr: 0.0005	train loss: 0.6524
epoch: 10	lr: 0.0005	train loss: 0.5622
epoch: 11	lr: 0.0005	train loss: 0.5614
epoch: 12	lr: 0.0005	train loss: 0.5175
epoch: 13	lr: 0.0005	train loss: 0.4982
epoch: 14	lr: 0.0005	train loss: 0.4404
epoch: 15	lr: 0.0005	train loss: 0.4185
epoch: 16	lr: 0.0005	train loss: 0.4270
epoch: 17	lr: 0.0005	train loss: 0.3891
epoch: 18	lr: 0.0005	train loss: 0.3461
epoch: 19	lr: 0.0005	train loss: 0.3734
epoch: 20	lr: 0.0005	train loss: 0.3084
epoch: 21	lr: 0.0005	train loss: 0.3358
epoch: 22	lr: 0.0005	train loss: 0.3181
epoch: 23	lr: 0.0005	train loss: 0.2858
epoch: 24	lr: 0.0005	train loss: 0.3459
epoch: 25	lr: 0.0005	train loss: 0.2658
epoch: 26

In [11]:
max_epoch = -1
max_val = 0.0
max_results = dict()

f = open(os.path.join(record_dir, 'split_{}_all.csv'.format(split)), 'w')

writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])

for epoch in range(1, cfg.max_epoch+1):
    print('======================EPOCH {}====================='.format(epoch))
    predict_backbone(model_name, model, model_dir, result_dir, features_path, vid_list_file_tst, 
                     epoch, actions_dict, device, sample_rate)    
    results = eval_txts(cfg.dataset_root, result_dir, dataset, split, model_name)
    
    writer.writerow([epoch, '%.4f'%(results['accu']), '%.4f'%(results['edit']),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[2])])])

    curr_val = sum([results[k] for k in results.keys()])
    max_val = max(max_val, curr_val)

    if curr_val == max_val:
        max_epoch = epoch
        max_results = results

print('EARNED MAXIMUM PERFORMANCE IN EPOCH {}'.format(max_epoch))
print(max_results)

f.close()

======================EPOCH 1=====================
Acc: 28.7288
Edit: 17.1729
F1@0.10: 11.0941
F1@0.25: 8.7316
F1@0.50: 5.6504
======================EPOCH 2=====================
Acc: 41.7966
Edit: 17.1408
F1@0.10: 11.9670
F1@0.25: 9.5823
F1@0.50: 6.3325
======================EPOCH 3=====================
Acc: 49.5860
Edit: 20.1660
F1@0.10: 14.7086
F1@0.25: 12.1310
F1@0.50: 7.8860
======================EPOCH 4=====================
Acc: 44.7704
Edit: 24.1434
F1@0.10: 15.2766
F1@0.25: 12.3478
F1@0.50: 8.3445
======================EPOCH 5=====================
Acc: 53.0941
Edit: 25.9354
F1@0.10: 18.0060
F1@0.25: 15.3753
F1@0.50: 10.6623
======================EPOCH 6=====================
Acc: 53.7027
Edit: 26.2027
F1@0.10: 17.7373
F1@0.25: 15.0517
F1@0.50: 10.5014
======================EPOCH 7=====================
Acc: 60.5330
Edit: 27.0354
F1@0.10: 19.1023
F1@0.25: 16.2006
F1@0.50: 11.5631
======================EPOCH 8=====================
Acc: 57.0461
Edit: 28.8578
F1@0.10: 21.2078
F1@0.25:

In [12]:
f = open(os.path.join(record_dir, 'split_{}_best.csv'.format(split)), 'w')
writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])
writer.writerow([max_epoch, '%.4f'%(max_results['accu']), '%.4f'%(max_results['edit']),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[2])])])
f.close()